In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# The <a href="https://en.wikipedia.org/wiki/DPLL_algorithm">Davis-Putnam Algorithm</a>

This notebook implements the algorithm of Davis and Putnam.  Further details about this algorithm are provided in the lecture notes.

The function `complement(l)` computes the complement of a literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{complement}(p) = \neg p$,
* $\texttt{complement}(\neg p) = p$.

As we are working with clauses that result form transforming given formulas into *conjunctive normal form* and these clauses do not contain $\top$ or $\bot$, we don't have to bother with $\top$ or $\bot$ in this function.

In [2]:
def complement(l):
    'Compute the complement of the literal L.'
    match l:
        case ('¬', p): return p
        case p       : return ('¬', p)

In [3]:
complement('p')

('¬', 'p')

In [4]:
complement(('¬', 'p'))

'p'

The function `extractVariable(l)` extracts the variable from the literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{extractVariable}(p) = p$,
* $\texttt{extractVariable}(\neg p) = p$.

In [5]:
def extractVariable(l):
    'Extract the propositional variable from the literal l.'
    match l:
        case ('¬', p): return p
        case p       : return p 

In [6]:
extractVariable('p')

'p'

In [7]:
extractVariable(('¬', 'p'))

'p'

The function `arb(S)` returns an arbitrary element from the set `S`.

In [8]:
def arb(S):
    'Return some member from the set S.'
    for x in S: 
        return x

In [9]:
import random as rnd

The function `selectVariable(Clauses, Forbidden)`
selects an arbitrary variable from a clause from the set `Clauses` that does not occur in the set `Forbidden`.  This variable is returned.

In [10]:
def selectVariable(Clauses, Forbidden):
    #return rnd.choice(list({ extractVariable(l) for C in Clauses for l in C } - Forbidden))
    return arb({ extractVariable(l) for C in Clauses for l in C } - Forbidden)

Given a set of clauses `Clauses` and a literal `l`, the procedure `reduce(Clauses, l)` performs all unit cuts and all unit subsumptions on clauses of of the set `Clauses` that are possible using the unit clause $\{\mathtt{l}\}$.  The resulting set of clauses is returned.  Mathematically, the function `reduce` is defined as follows:
$$\texttt{reduce}(\texttt{Clauses},l)  := 
 \Bigl\{\, C \backslash \bigl\{\overline{\,l\,}\bigr\} \;|\; C \in \texttt{Clauses} \wedge \overline{\,l\,} \in C \,\Bigr\} 
       \,\cup\, \Bigl\{\, C \in \texttt{Clauses} \mid \overline{\,l\,} \not\in C \wedge l \not\in C \Bigr\} \cup \bigl\{\{l\}\bigr\}.
$$
This function should only be called if the unit clause $\{l\}$ is an element of the set `Clauses`.

In [11]:
def reduce(Clauses, l):
    lBar = complement(l)  
    return   { C - { lBar } for C in Clauses if lBar     in C                }  \
           | { C            for C in Clauses if lBar not in C and l not in C }  \
           | { frozenset({l}) }

`Clauses` is a set of clauses.  The call `saturate(Clauses)` computes the set of those clauses that can be derived from `Clauses` via repeated applications of unit cuts or unit subsumptions.

In [12]:
def saturate(Clauses):
    S     = Clauses.copy()
    Units = { C for C in S if len(C) == 1 } # set of unit clauses occurring in C
    Used  = set()                           # remember which unit clauses have already been used
    while len(Units) > 0:  # iterate as long as we derive new unit clauses
        unit  = Units.pop()
        Used |= { unit }
        l     = arb(unit)
        S     = reduce(S, l)
        Units = { C for C in S if len(C) == 1 } - Used        
    return S

The function `isFalsum(Clauses)` is true if and only if `Clauses` is a set of clauses that contains nothing but the empty clause `{}`. 

In [13]:
def isFalsum(Clauses):
    return Clauses == { frozenset() }

The function `solve(Clauses, Variables)` takes a set of clauses and a set of variables as input.  The function tries to compute a variable assignment that satisfies all clauses in `Clauses`.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If the set `Clauses` is unsatisfiable, then the set `{{}}` is returned instead.

The argument `Variables` is a set containing all those variables that have already been used to reduce the clauses.  Initially, this set is empty.

In [14]:
def solve(Clauses, Variables):
    S      = saturate(Clauses)
    empty  = frozenset()
    Falsum = {empty}
    if empty in S:                  # S is inconsistent
        return Falsum           
    if all(len(C) == 1 for C in S): # S is trivial
        return S
    # case distinction on variable p
    p      = selectVariable(S, Variables)
    Result = solve(S | { frozenset({p}) },  Variables | { p })
    if not isFalsum(Result):
        return Result
    return solve(S | { frozenset({complement(p)}) }, Variables | { p })

The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets.  This is only used for pretty printing.

In [15]:
def literal_to_str(C):
    'Convert a unit clause to a string.'
    l = arb(C)
    if l[0] == '¬':
        return f'{str(l[1])} ↦ False' 
    else:
        return f'{str(l)} ↦ True'
    
def toString(S, Simplified):
    'Convert the set S of frozen sets to a string where frozen sets are written as sets.'
    if len(Simplified) == 1:
        Clause = arb(Simplified)
        if len(Clause) == 0:
            return f'{prettify(S)} is unsolvable'
    else:
        result = '{ ' + ', '.join({ literal_to_str(C) for C in Simplified }) + ' }'
        return result

In [16]:
def prettify(Clauses):
    return ', '.join({str(set(C)) for C in Clauses})

In [17]:
c1 = frozenset({ 'r', 'p', 's' })
c2 = frozenset({ 'r', 's' })
c3 = frozenset({ 'p', 'q', 's' })
c4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6 = frozenset({ 'p', ('¬', 'q'), 'r'})
c7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8 = frozenset({ ('¬', 'p'), ('¬', 's')})
c9 = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
c0 = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
S  = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 }
print(toString(S, solve(S, set())))

{ p ↦ False, s ↦ True, q ↦ False, r ↦ False }


In [18]:
c10 = frozenset({ 'p', 'r', 'q', ('¬', 's') })
S   = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10 }
print(toString(S, solve(S, set())))

{('¬', 'q'), ('¬', 'p')}, {('¬', 'q'), 'r', 'p'}, {'s', 'q', 'p'}, {('¬', 's'), 'q', 'r', ('¬', 'p')}, {'s', 'r', 'p'}, {('¬', 's'), ('¬', 'r'), 'q'}, {'s', 'r'}, {('¬', 's'), ('¬', 'p')}, {('¬', 's'), 'q', 'r', 'p'}, {('¬', 'r'), ('¬', 'q'), 'p'}, {'s', ('¬', 'r'), ('¬', 'p')} is unsolvable


In [19]:
c1 = frozenset({ 'r', 'p', 's' })
c2 = frozenset({ 'r', 's' })
c3 = frozenset({ 'q', 'p', 's' })
c4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6 = frozenset({ 'p', ('¬', 'q'), 'r' })
c7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8 = frozenset({ 'p', 'q', 'r', 's' })
c9 = frozenset({ 'r', ('¬', 's'), 'q' })
c10 = frozenset({ 's', ('¬', 'r'), ('¬', 'q') })
c11 = frozenset({ 's', ('¬', 'r') })
c12 = frozenset({ 'r', ('¬', 's') })
S  = {c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12}
solve(S, set())

{frozenset({'q'}), frozenset({'r'}), frozenset({'s'}), frozenset({('¬', 'p')})}